In [39]:
#!/usr/bin/env python3
#!/usr/bin/python3
import os
import os.path
import fileinput
import re
import subprocess
from astropy import coordinates as coord
import sys
from astroquery.heasarc import Heasarc
from astropy.coordinates import SkyCoord
from astropy.time import Time
import numpy as np
import shutil
import glob
import warnings
import heasoftpy as hsp
heasarc = Heasarc()
os.system("export HEADASNOQUERY= \nexport HEADASPROMPT=/dev/null")

0

In [40]:
counts=[]
count_rate=[]
bkg_counts=[]
bkg_count_rate=[]
fluence=[]
countrate_xspec=[]
file_name=[]
mjd=[]
unc=[]
res=[]
counts_xspec=[]

In [41]:
#### Change these values#####
name = "wr138"
mission = 'swiftmastr'
path_folder = "/home/idies/workspace/Storage/casciotk/persistent/SWIFT"
ap = 45 #aperture
back_in = 170
back_out = 200
#############################

In [42]:
coords = coord.SkyCoord.from_name(name, frame="icrs")
ra = coords.ra.deg
dec = coords.dec.deg
cols = heasarc.query_mission_cols(mission=mission)
#table = heasarc.query_region(coords,mission=mission,radius='1 arcmin')

In [43]:
#new rmf code ---------------------
    
paths = []
file_paths = []
    
paths_txt = """
/FTP/caldb/data/swift/xrt/cpf/rmf/
"""
paths = paths_txt.split('\n')[1:-1]
glob.glob(f'{paths[0]}/*')
file_paths = glob.glob(f'{paths[0]}/swxpc0to12s6_20130101v014.rmf')
destination_path = "."
shutil.copy(file_paths[0], destination_path)

os.system("mv swxpc0to12s6_20130101v014.rmf PC.rmf")


0

In [44]:
rad = 0
count = 0
index = 0
table = heasarc.query_region(coords,mission=mission,radius= ('11 arcmin'),sortvar='OBSID')

for obs in range (0, len(table['OBSID']), 1): 
    warnings.filterwarnings("ignore")
    print(table['OBSID'][obs])
    obsid = table['OBSID'][obs]
    dates = table['START_TIME']
    
    warnings.filterwarnings("default") #making the warnings come back
    
    #add fields='OBSID,START_TIME,NAME,RA,DEC,POINTING_MODE,EXPOSURE')
    #table=table_tot[np.where((table_tot["START_TIME"] > 57177.0)&(table_tot["START_TIME"] < 57997.5))]
    #table_tot[np.where((table_tot["START_TIME"])>57000.)]
    #convert MJD to year_mm format#

    os.chdir(path_folder)

    #make a directory if it doesn't already exist
    if os.path.isdir(name) == False:
        os.mkdir(name)
        print('new folder created')

    path = path_folder + '/' + name

    os.chdir(path)
    #dates=table['START_TIME']
    #obsid=table['OBSID']
    #type(dates.data) # lists the astropy column as an array
    datess=Time(dates.data,format='mjd')
    datess.format='fits'
    yrmm=[x[0:7] for x in datess.value] # retrieve the yyyy-mm part from the string
    datess=[x.replace("-","_") for x in yrmm] # the wget command needs "_" not "-"
    date = datess[obs]
    
    print(obsid)
    print(date)

    filename="download_{}.sh".format(name) #name of the file
    f=open(filename,"w")
    
    '''
    for i in range(len(table)):
        f.write(("wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks" 
        " https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{}//{}/xrt/".format(date[i],obsid[i]))+"\n")
        f.write(("wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks" 
        " https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{}//{}/auxil/".format(date[i],obsid[i]))+"\n")
    '''
    
    f.write(("wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks" 
    " https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{}//{}/xrt/".format(date,obsid))+"\n")
    f.write(("wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks" 
    " https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{}//{}/auxil/".format(date,obsid))+"\n")
    
    f.close()
    os.system("chmod a+x {}".format(filename)) # change the permissions
    os.system("./{}".format(filename)) # run the script

    ## running the pipeline and analysis now

    for dirpath, dirnames, filenames in os.walk(path):
        for filename0 in [f for f in filenames 
        if f.endswith(("xpcw2po_cl.evt.gz","xpcw3po_cl.evt.gz","xpcw4po_cl.evt.gz"))]:
            os.chdir(path)
            dir=filename0[2:13]
            filename=filename0[0:27]+'.gz'
            print(filename)
            print(dir)
            #cmd=f'xrtpipeline indir={path}/{dir} outdir={path_folder}/{name}"-output"/{dir}/xrt/event srcra={ra} srcdec={dec} steminputs={dir} createexpomap=yes clobber=yes'
            #os.system(cmd)
            hsp.xrtpipeline(indir=f"{path}/{dir}", outdir=f"{path_folder}/{name}-output/{dir}/xrt/event", srcra=ra, srcdec=dec, steminputs=f'sw{dir}', createexpomap=True, clobber=True, noprompt=True)
            outpath=f'{path_folder}/{name}-output/{dir}'
            os.chdir(f'{outpath}/xrt/event')
            file=open("pc_src.reg","w")
            file.write(f'global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n fk5 \n')
            file.write(f'circle({ra},{dec},20")')
            file.close()
            filename=filename0[0:27]
            mjd_i=subprocess.check_output(f'fkeyprint {filename}[1] MJD-OBS \n',shell=True)
            mjd.append(str(mjd_i).split("n")[5][10:31])
            cmd="xselect <<EOF\nxel\nread event\n./\n"+str(filename)+"\nyes\nfilter pha_cutoff 30 1000\nextract image\nsave image image.img\nyes\nexit\nno\nEOF\n"
            os.system(cmd)


    #       cmd=f'xrtpipeline indir={path}/{dir} outdir={path}/{dir}/xrt/event srcra={ra} srcdec={dec} steminputs={dir} createexpomap=yes clobber=yes'
            #print(cmd)

            """
            %%only for centroid finding%%
            center="xrtcentroid infile=image.img outfile=pos.txt outdir=./ interactive=no boxra='18 44 41.6' boxdec='-03 05 35.97' boxradius=4 calcpos=yes clobber=yes"
            os.system(center)
            xpos=[]
            ypos=[]
            with open("pos.txt","r") as pos:
                for line in pos:
                    if line.startswith("X"):
                        xpos.append(float((line.strip().split()[2])))
                    elif line.startswith("Y"):
                        ypos.append(float((line.strip().split()[2])))
            for line in fileinput.FileInput("pc_src.reg", inplace=1):
                line=line.replace('circle(500,500,45)','circle('+str(xpos[0])+','+str(ypos[0])+',45)')
                print(line)
            """

    pileup_check="xselect <<EOF\nxsel\nread event\n./\n"+str(filename)+"\nyes\ncpd /xw\nfilter region pc_src.reg\nextract spectrum\nexit\nno\nEOF\n"
    os.system(pileup_check)
    with open("xselect.log","r") as f:
        for line in f:
            if line.startswith(" Spectrum         has"):
                counts=(line.strip().split()[5])
                res.append(float(counts))
                pix_Scale=2.36 # arcsec 2.36 arcsec =1 pixel
                if (1.0>= float(counts) > 0.5):
                    inner=2*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (3.0>= float(counts) > 1.0):
                    inner=4*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (4.0>= float(counts) > 3.0):
                    inner=5*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (6.0>= float(counts) > 4.0):
                    inner=6*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (8.0>= float(counts) > 6.0): #4.5 to 9 - 7 pixel
                    inner=7*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (10.>= float(counts) > 8.0):# 10 and 12 - 11 pix
                    inner=9*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (float(counts) > 10.): 
                    inner=12*pix_Scale
                    for line in fileinput.FileInput("pc_src.reg", inplace=1):
                        line=line.replace(f'circle({ra},{dec},{ap}")',f'annulus({ra},{dec},{inner}",{ap}")')
                        print(line)
                elif (0.5>= float(counts) > 0.0):
                    continue

    f.close()
            #no time filtering needed for pc mode
    bfile=open("pc_back.reg","w")
    bfile.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1'+"\n")
    bfile.write("image"+"\n")
    bfile.write(f'annulus({ra} {dec}),{back_in}",{back_out}")')
    bfile.close()
    cmdf="xselect <<EOF\nxel\nread event\n./\n"+str(filename)+"\nyes\nfilter region pc_src.reg\nextract spectrum \nsave spectrum spec\nyes\nextract curve\nset binsize 10\nextract curve\nsave curve pc_lc\nyes\n"
    cmdf+="clear region\nfilter region pc_back.reg\nextract curve\nset binsize 10\nextract curve\nsave curve pc_lcback\nyes\nextract spectrum\nsave spectrum back\nyes\nexit\nno\nEOF\n"
    os.system(cmdf)
    os.system(f'xrtmkarf outfile=PC.arf expofile= {filename[0:20]}_ex.img phafile=spec.pha srcx=-1 srcy=-1 psfflag=yes clobber=yes > arf.log')

    '''
    #new rmf code ---------------------
    
    paths = []
    file_paths = []
    
    paths_txt = """
    /FTP/caldb/data/swift/xrt/cpf/rmf/
    """
    paths = paths_txt.split('\n')[1:-1]
    print(paths[0])
    glob.glob(f'{paths[0]}/*')
    file_paths = glob.glob(f'{paths[0]}/swxpc0to12s6_20130101v014.rmf')
    destination_path = "."
    shutil.copy(file_paths[0], destination_path)

    os.system("mv swxpc0to12s6_20130101v014.rmf PC.rmf")

    #old rmf code ---------------------
    '''
    """     
    for line in open("arf.log","r"):
        if "ON AVERAGE" in line:
            a=[float(x) for x in re.findall(re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\*[0-9]+)?'), line)]
            fluence.append(a[0])     
        if "~/home/idies/workspace/Storage/casciotk/persistent/SWIFT/RMF" in line:   #if "/data/swift/xrt/cpf/rmf" in line:
            print("wget "+str(line[87:116])+"")
            os.system("wget http://localhost:8000/" +str(line[87:116]))
            os.system("mv sw*rmf PC.rmf") # same rmf for this

    """

    grp_cmd="grppha <<EOF\nspec.pha\n!fspec.pha\nchkey BACKFILE back.pha\nchkey RESPFILE PC.rmf\nchkey ANCRFILE PC.arf\ngroup min 20\nexit\nEOF\n"
    os.system(grp_cmd)
    grp_cmd_nogroup="grppha <<EOF\nspec.pha\n!fspec_cstat.pha\nchkey BACKFILE back.pha\nchkey RESPFILE PC.rmf\nchkey ANCRFILE PC.arf\ngroup min 1\nexit\nEOF\n"
    os.system(grp_cmd_nogroup)
    file_name.append(filename)
    #write counts from xspec
    new=os.getcwd()
    #NOTE FOR KATIE: WORK FROM HERE and look at lines 240-243 below
    #for line in out.split("\n"):
    #    counts_xspec.append(float(line.split(" ")))
    
    counts_xspec = []
    
    xspec_cts="xspec <<EOF\ndata spec.pha\nexit\nEOF" #changed from fspec.pha
    print ("xspec_cts output: ", xspec_cts)
    out=subprocess.check_output(xspec_cts,shell=True).decode()
    
    counts_xspec.append((out.split("\n")[9][39:]))
    print("\n\n")
    print(counts_xspec)

    #gives the count and error, might have to split them into different variables. 
    #save the count and error
    #somethings going wrong here, play around with the for statement above. 
    #print(out("\n")[9][39:]) is printing the count and error correctly
    '''
    os.chdir(path)
    with open("pileup_counts_new.txt","w") as f:
        f.write("ID,MJD,piled_up_cts,fluence,xspec_counts\n")
        for i in range(len(res)):
            f.write(f'{file_name[i]},{mjd[i]},{res[i]},{fluence[i]},{counts_xspec[i]} \n')
    '''        
    #edited code to append a file instead of making a new one

    os.chdir(path_folder)
    with open("pileup_counts_" + name + ".txt","a") as f:
        f.write("\n\n Name,ID,MJD,piled_up_cts,xspec_counts\n")
        for i in range(len(res)):
            f.write(f'{name},{obsid},{mjd[i]},{res[i]},{counts_xspec[0]}\n\n')
        f.close()


<>:185: DeprecationWarning: invalid escape sequence \ 
<>:185: DeprecationWarning: invalid escape sequence \ 
/tmp/ipykernel_303/182249945.py:185: DeprecationWarning: invalid escape sequence \ 
  """


00040153001
00040153001
2011_03
sw00040153001xpcw3po_cl.evt.gz
00040153001
 
                         **  XSELECT V2.5b  **
 
> Enter session name >[xsel7020] xel
xel:SUZAKU > read event
> Enter the Event file dir >[/home/idies/workspace/Storage/casciotk/persistent/SWIFT/wr138-output/00040153001/xrt/event//] ./
> Enter Event file list >[sw00040153001xwtw2st_cl.evt] sw00040153001xpcw3po_cl.evt
Got new mission: SWIFT
> Reset the mission ? >[yes] yes
 
 
        OBJECT           OBS_ID      DATE-OBS            DATAMODE
      1 J201700.4+372524 00040153001 2011-03-14T20:23:57 PHOTON
 
 Notes: XSELECT set up for      SWIFT
 Time keyword is TIME       in units of s
 Default timing binsize =   5.0000
 
Setting...
 Image  keywords   = X          Y           with binning =    1
 WMAP   keywords   = X          Y           with binning =    1
 Energy keyword   = PI                     with binning =    1
 
Getting Min and Max for Energy Column...
Got min and max for PI:     0   1023
 
Got the min